# 🔧 Autoencoder Básico: Implementação e Treinamento

**Tutorial de Espaço Latente - Notebook 2**

Neste notebook, vamos implementar e treinar um Autoencoder real no dataset MNIST.

## 🎯 Objetivos
- Entender a arquitetura de um Autoencoder
- Treinar o modelo no MNIST
- Visualizar o espaço latente 2D
- Analisar reconstruções
- Explorar interpolações

In [ ]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt

# Importa nossos módulos
from src.models.autoencoder import Autoencoder
from src.utils.data_loader import load_mnist
from src.utils.training import train_model
from src.utils.visualization import (
    visualize_latent_space,
    plot_reconstructions,
    plot_interpolation,
    plot_training_history
)

# Configuração
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

## 📊 Passo 1: Carregando os Dados

O MNIST contém 70.000 imagens de dígitos manuscritos (0-9), cada uma com 28x28 pixels.

In [ ]:
# Carrega MNIST
train_loader, val_loader, test_loader = load_mnist(batch_size=128)

print(f"Train samples: {len(train_loader.dataset)}")
print(f"Val samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")

# Visualiza algumas imagens
images, labels = next(iter(train_loader))

fig, axes = plt.subplots(2, 10, figsize=(15, 3))
for i in range(20):
    ax = axes[i//10, i%10]
    ax.imshow(images[i].squeeze(), cmap='gray')
    ax.set_title(f'{labels[i].item()}', fontsize=10)
    ax.axis('off')
plt.suptitle('MNIST Dataset Samples', fontweight='bold')
plt.tight_layout()
plt.show()

## 🏗️ Passo 2: Criando o Autoencoder

Nossa arquitetura:
```
Input (784) → [512] → [256] → [128] → Latent (2) → [128] → [256] → [512] → Output (784)
            Encoder                                           Decoder
```

Taxa de compressão: 784 / 2 = **392x**!

In [ ]:
# Cria modelo
model = Autoencoder(
    input_dim=784,
    latent_dim=2,  # 2D para visualização
    hidden_dims=[512, 256, 128]
)

print(model)
print(f"\nCompression ratio: {model.compression_ratio():.1f}x")

# Conta parâmetros
n_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {n_params:,}")

## 🎓 Passo 3: Treinamento

Vamos treinar o modelo para minimizar o erro de reconstrução (MSE).

In [ ]:
# Treina
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    learning_rate=1e-3,
    device=DEVICE,
    early_stopping_patience=5,
    verbose=True
)

# Plota curvas de treinamento
plot_training_history(history)

## 📈 Passo 4: Visualizando o Espaço Latente

Vamos ver como o modelo organizou os dígitos no espaço 2D.

In [ ]:
# Visualiza espaço latente
visualize_latent_space(model, test_loader, device=DEVICE,
                      title='Autoencoder Latent Space (MNIST)')

**Análise:**
- Dígitos similares ficam próximos no espaço latente
- Clusters claramente separados
- Apenas 2 dimensões capturam a essência de 784 pixels!

## 🔍 Passo 5: Analisando Reconstruções

Quão bem o modelo reconstrói as imagens originais?

In [ ]:
# Plota reconstruções
plot_reconstructions(model, test_loader, n_samples=10, device=DEVICE)

## 🌊 Passo 6: Interpolação no Espaço Latente

O que acontece quando "caminhamos" entre dois dígitos no espaço latente?

In [ ]:
# Pega dois dígitos
data, labels = next(iter(test_loader))

# Interpola entre primeiro e segundo dígito
plot_interpolation(model, data[0], data[5], num_steps=10, device=DEVICE)

print(f"\nInterpolating from digit {labels[0]} to digit {labels[5]}")

## 🎨 Passo 7: Explorando Manualmente o Espaço Latente

Vamos decodificar pontos arbitrários do espaço latente.

In [ ]:
model.eval()

# Define pontos no espaço latente
latent_points = torch.tensor([
    [0, 0],    # Centro
    [-2, -2],  # Canto inferior esquerdo
    [2, 2],    # Canto superior direito
    [-2, 2],   # Canto superior esquerdo
    [2, -2],   # Canto inferior direito
    [0, 3],    # Topo
    [3, 0],    # Direita
], dtype=torch.float32).to(DEVICE)

# Decodifica
with torch.no_grad():
    decoded = model.decode(latent_points)
    decoded = decoded.view(-1, 28, 28).cpu()

# Visualiza
fig, axes = plt.subplots(1, 7, figsize=(14, 2))
for i, ax in enumerate(axes):
    ax.imshow(decoded[i], cmap='gray')
    ax.set_title(f'z={latent_points[i].cpu().numpy()}', fontsize=8)
    ax.axis('off')
plt.suptitle('Decoded Images from Latent Points', fontweight='bold')
plt.tight_layout()
plt.show()

## 📊 Passo 8: Análise Quantitativa

In [ ]:
from src.utils.training import evaluate_model

# Avalia no test set
test_metrics = evaluate_model(model, test_loader, device=DEVICE, is_vae=False)

print("Test Set Metrics:")
print(f"  Loss (MSE): {test_metrics['loss']:.6f}")
print(f"\nInterpretation:")
print(f"  - Average pixel error: {np.sqrt(test_metrics['loss']):.4f}")
print(f"  - Compression: 784 → 2 dimensions ({model.compression_ratio():.0f}x)")

## 🔬 Experimento: Diferentes Dimensões Latentes

O que acontece com dimensões latentes maiores?

In [ ]:
# Testa diferentes latent_dims
latent_dims = [2, 5, 10, 20, 50]
results = []

for latent_dim in latent_dims:
    print(f"\nTraining with latent_dim={latent_dim}...")
    
    model_temp = Autoencoder(input_dim=784, latent_dim=latent_dim)
    
    history_temp = train_model(
        model=model_temp,
        train_loader=train_loader,
        val_loader=val_loader,
        num_epochs=10,
        device=DEVICE,
        verbose=False
    )
    
    final_loss = history_temp['val_loss'][-1]
    results.append((latent_dim, final_loss))
    print(f"  Final val loss: {final_loss:.6f}")

# Plota
dims, losses = zip(*results)
plt.figure(figsize=(10, 6))
plt.plot(dims, losses, marker='o', linewidth=2, markersize=10)
plt.xlabel('Latent Dimension', fontsize=12, fontweight='bold')
plt.ylabel('Validation Loss', fontsize=12, fontweight='bold')
plt.title('Reconstruction Quality vs Latent Dimension', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Observação:")
print("- Dimensões maiores → melhor reconstrução")
print("- Mas perde interpretabilidade e compressão")
print("- Trade-off entre compressão e qualidade")

## 📝 Resumo

Neste notebook, aprendemos:

✅ Como treinar um Autoencoder em PyTorch  
✅ Visualizar o espaço latente aprendido  
✅ Analisar reconstruções  
✅ Interpolar no espaço latente  
✅ Trade-off entre compressão e qualidade  

**Limitações do Autoencoder:**
- Espaço latente não é contínuo (buracos)
- Não pode gerar novas amostras facilmente
- Aprende mapeamento determinístico

**Solução:** Variational Autoencoder (VAE)!

---

## 🚀 Próximo Notebook

No **Notebook 03**, vamos explorar VAEs que resolvem essas limitações!

→→ `03_vae_explicativo.ipynb`